In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/TLCN/Code/hybrid_movie_v2/hybrid_movie_v2')

In [4]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163008 sha256=d1525be1f117cd7dce23a4a84a57826d9c3350a79276ab1e5de38589e0635f5a
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


### Load file

In [5]:
!python "MovieLens.py"
!python "NCF.py"
!python "HybridAlgorithm.py"
!python "SGD.py"
!python "ContentLSTMModel.py"
!python "ContentKNNAlgorithm.py"

2024-02-17 06:36:07.303042: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 06:36:07.303123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 06:36:07.304669: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-17 06:36:07.313478: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 06:36:08.863991: W tensorflow/compiler/tf2

### Import library

In [6]:
from MovieLens import MovieLens
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, KNNWithMeans
from NCF import NCF
from SGD import SGD
import random
import numpy as np
import pandas as pd
from surprise.model_selection import train_test_split
from surprise import accuracy
from ContentKNNAlgorithm import ContentKNNAlgorithm
from ContentLSTMModel import ContentLSTMAlgorithm
from time import time

### Train model

In [7]:
# Function to train and test a model while measuring the execution time
def train_and_test_model(model, trainset, testset):
    start_time = time()  # Record start time
    model.fit(trainset)  # Train the model
    end_time = time()  # Record end time for training
    train_time = end_time - start_time  # Calculate training time

    start_time = time()  # Record start time for testing
    predictions = model.test(testset)  # Test the model
    end_time = time()  # Record end time for testing
    test_time = end_time - start_time  # Calculate testing time

    # Calculate RMSE and MAE
    rmse = accuracy.rmse(predictions, verbose=False)
    mae = accuracy.mae(predictions, verbose=False)

    return rmse, mae, train_time, test_time

### Load data

In [8]:
def LoadMovieLensData():
    ml = MovieLens()
    data = ml.loadMovieLensLatestSmall()
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

(ml, evaluationData, rankings) = LoadMovieLensData()

trainset, testset =  train_test_split(evaluationData, test_size=.3, random_state=0)

result_df = pd.DataFrame(columns=['Algorithm', 'RMSE', 'MAE', 'Training Time (s)', 'Testing Time (s)'])

### Content based

KNN

In [9]:
# Content based - KNN
ContentKNN = ContentKNNAlgorithm()
content_knn_rmse, content_knn_mae, content_knn_train_time, content_knn_test_time = train_and_test_model(ContentKNN, trainset, testset)
result_content_knn = {
    'Algorithm': 'ContentKNN',
    'RMSE': content_knn_rmse,
    'MAE': content_knn_mae,
    'Training Time (s)': content_knn_train_time,
    'Testing Time (s)': content_knn_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_content_knn])], ignore_index=True)


Computing content-based similarity matrix...


In [10]:
result_df

,Algorithm,RMSE,MAE,Training Time (s),Testing Time (s)
0,ContentKNN,0.90161,0.694323,341.447368,12.421732


LSTM

In [ ]:
# Content based - LSTM
ContentLSTM = ContentLSTMAlgorithm()
content_lstm_rmse, content_lstm_mae, content_lstm_train_time, content_lstm_test_time = train_and_test_model(ContentLSTM, trainset, testset)
result_content_lstm = {
    'Algorithm': 'ContentLSTM',
    'RMSE': content_lstm_rmse,
    'MAE': content_lstm_mae,
    'Training Time (s)': content_lstm_train_time,
    'Testing Time (s)': content_lstm_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_content_lstm])], ignore_index=True)


Epoch 1/100
3530/3530 [==============================] - 393s 108ms/step - loss: 2.7577 - val_loss: 1.4993 - lr: 1.0000e-04
Epoch 2/100
3530/3530 [==============================] - 360s 102ms/step - loss: 1.7805 - val_loss: 1.2386 - lr: 1.0000e-04
Epoch 3/100
3530/3530 [==============================] - 364s 103ms/step - loss: 1.3841 - val_loss: 1.0897 - lr: 1.0000e-04
Epoch 4/100
3530/3530 [==============================] - 359s 102ms/step - loss: 1.1743 - val_loss: 1.0236 - lr: 1.0000e-04
Epoch 5/100
3530/3530 [==============================] - 359s 102ms/step - loss: 1.0995 - val_loss: 0.9852 - lr: 1.0000e-04
Epoch 6/100
3530/3530 [==============================] - 362s 103ms/step - loss: 1.0698 - val_loss: 0.9433 - lr: 1.0000e-04
Epoch 7/100
3530/3530 [==============================] - 359s 102ms/step - loss: 1.0477 - val_loss: 0.9512 - lr: 1.0000e-04
Epoch 8/100
3530/3530 [==============================] - 360s 102ms/step - loss: 1.0331 - val_loss: 0.9300 - lr: 1.0000e-04
Epoch 9/

In [ ]:
result_df

### Colaborative filtering

#### User - User Collaborative Filtering (User based)

In [ ]:
# Collaborative filtering - User based
UserKNNmeans = KNNWithMeans(sim_options={'name': 'cosine', 'user_based': True})
userBased_rmse, userBased_mae, userBased_train_time, userBased_test_time = train_and_test_model(UserKNNmeans, trainset, testset)
result_userBased = {
    'Algorithm': 'User Based',
    'RMSE': userBased_rmse,
    'MAE': userBased_mae,
    'Training Time (s)': userBased_train_time,
    'Testing Time (s)': userBased_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_userBased])], ignore_index=True)


In [ ]:
result_df

#### Item - Item Collaborative Filtering (Item based)

In [ ]:
# Collaborative filtering - Item based
ItemKNNmeans = KNNWithMeans(sim_options={'name': 'cosine', 'user_based': False})
itemBased_rmse, itemBased_mae, itemBased_train_time, itemBased_test_time = train_and_test_model(ItemKNNmeans, trainset, testset)
result_itemBased = {
    'Algorithm': 'Item Based',
    'RMSE': itemBased_rmse,
    'MAE': itemBased_mae,
    'Training Time (s)': itemBased_train_time,
    'Testing Time (s)': itemBased_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_itemBased])], ignore_index=True)


In [ ]:
result_df

#### SVD Untuned

In [ ]:
# SVD Untuned
svdUntuned_model = SVD()
svdUntuned_rmse, svdUntuned_mae, svdUntuned_train_time, svdUntuned_test_time = train_and_test_model(svdUntuned_model, trainset, testset)
result_svdUntuned = {
    'Algorithm': 'SVD Untuned',
    'RMSE': svdUntuned_rmse,
    'MAE': svdUntuned_mae,
    'Training Time (s)': svdUntuned_train_time,
    'Testing Time (s)': svdUntuned_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_svdUntuned])], ignore_index=True)


In [ ]:
result_df

#### SVD Tuned

In [ ]:
# SVD Tuned
svdTuned_model = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)
svdTuned_rmse, svdTuned_mae, svdTuned_train_time, svdTuned_test_time = train_and_test_model(svdTuned_model, trainset, testset)
result_svdTuned = {
    'Algorithm': 'SVD Tuned',
    'RMSE': svdTuned_rmse,
    'MAE': svdTuned_mae,
    'Training Time (s)': svdTuned_train_time,
    'Testing Time (s)': svdTuned_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_svdTuned])], ignore_index=True)


In [ ]:
result_df

#### SGD

In [ ]:
# SGD - Uncomment if needed
# sgd_model = SGD(learning_rate=0.01, n_epochs=10, n_factors=10)
# sgd_rmse, sgd_mae, sgd_train_time, sgd_test_time = train_and_test_model(sgd_model, trainset, testset)
# result_sgd = {
#     'Algorithm': 'SGD',
#     'RMSE': sgd_rmse,
#     'MAE': sgd_mae,
#     'Training Time (s)': sgd_train_time,
#     'Testing Time (s)': sgd_test_time
# }
# result_df = result_df.concat(result_sgd, ignore_index=True)

In [ ]:
# sgd_model = SGD(learning_rate=0.01, n_epochs=10, n_factors=10)
# sgd_model.fit(trainset)

In [ ]:
# sgd_test = sgd_model.test(testset)

In [ ]:
# sgd_rmse = accuracy.rmse(sgd_test, verbose=False)
# sgd_mae = accuracy.mae(sgd_test, verbose=False)

# result_sgd = {
#     'Algorithm': 'SGD',
#     'RMSE': sgd_rmse,
#     'MAE': sgd_mae
# }

# result_df.loc[len(result_df)] = result_sgd

In [ ]:
# print(f'Mean Absolute Error (MAE) on test data: {sgd_mae}')
# print(f'Root Mean Squared Error (RMSE) on test data: {sgd_rmse}')

In [ ]:
# result_df

#### Neural Collaborative Filtering (NCF)

##### Transform data

In [ ]:
trainset_test = trainset.build_testset()
train_data = pd.DataFrame(trainset_test, columns=['userId', 'movieId', 'rating'])
test_data= pd.DataFrame(testset, columns=['userId', 'movieId', 'rating'])

In [ ]:
n_users, n_movies = ml.get_number()

start_time = time()
ncf_model = NCF(n_users,n_movies)
ncf_model.fit(train_data)
end_time = time()
ncf_train_time = end_time - start_time

start_time = time()
ncf_mae, ncf_rmse = ncf_model.test(test_data)
end_time = time()
ncf_test_time = end_time - start_time

In [ ]:
result_ncf = {
    'Algorithm': 'Neural',
    'RMSE': ncf_rmse,
    'MAE': ncf_mae,
    'Training Time (s)': ncf_train_time,
    'Testing Time (s)': ncf_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_ncf])], ignore_index=True)
result_df

### Hybrid

##### ContentLSTM - Userbased

In [ ]:
# Hybrid Content Based - User Based
content_userBased_model = HybridAlgorithm([UserKNNmeans.fit(trainset), ContentLSTM.fit(trainset)], [0.7, 0.3])
content_userBased_rmse, content_userBased_mae, content_userBased_train_time, content_userBased_test_time = train_and_test_model(content_userBased_model, trainset, testset)
result_content_userBased = {
    'Algorithm': 'Hybrid Content Based LSTM - Userbased',
    'RMSE': content_userBased_rmse,
    'MAE': content_userBased_mae,
    'Training Time (s)': content_userBased_train_time,
    'Testing Time (s)': content_userBased_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_content_userBased])], ignore_index=True)


In [ ]:
result_df

##### ContentKNN - Userbased

In [ ]:
# Hybrid Content Based KNN - User Based
contentknn_userBased_model = HybridAlgorithm([UserKNNmeans.fit(trainset), ContentKNN.fit(trainset)], [0.7, 0.3])
contentknn_userBased_rmse, contentknn_userBased_mae, contentknn_userBased_train_time, contentknn_userBased_test_time = train_and_test_model(contentknn_userBased_model, trainset, testset)
result_contentknn_userBased = {
    'Algorithm': 'Hybrid Content Based KNN - Userbased',
    'RMSE': contentknn_userBased_rmse,
    'MAE': contentknn_userBased_mae,
    'Training Time (s)': contentknn_userBased_train_time,
    'Testing Time (s)': contentknn_userBased_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_contentknn_userBased])], ignore_index=True)


In [ ]:
result_df

##### ContentLSTM - Itembased

In [ ]:
# Hybrid Content Based - Item Based
content_itemBased_model = HybridAlgorithm([ContentLSTM.fit(trainset), ItemKNNmeans.fit(trainset)], [0.5, 0.5])
content_itemBased_rmse, content_itemBased_mae, content_itemBased_train_time, content_itemBased_test_time = train_and_test_model(content_itemBased_model, trainset, testset)
result_content_itemBased = {
    'Algorithm': 'Hybrid Content Based LSTM - Itembased',
    'RMSE': content_itemBased_rmse,
    'MAE': content_itemBased_mae,
    'Training Time (s)': content_itemBased_train_time,
    'Testing Time (s)': content_itemBased_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_content_itemBased])], ignore_index=True)


In [ ]:
result_df

##### ContentKNN - Itembased

In [ ]:
# Hybrid Content Based KNN - Item Based
contentknn_itemBased_model = HybridAlgorithm([ContentKNN.fit(trainset), ItemKNNmeans.fit(trainset)], [0.5, 0.5])
contentknn_itemBased_rmse, contentknn_itemBased_mae, contentknn_itemBased_train_time, contentknn_itemBased_test_time = train_and_test_model(contentknn_itemBased_model, trainset, testset)
result_contentknn_itemBased = {
    'Algorithm': 'Hybrid Content Based KNN - Itembased',
    'RMSE': contentknn_itemBased_rmse,
    'MAE': contentknn_itemBased_mae,
    'Training Time (s)': contentknn_itemBased_train_time,
    'Testing Time (s)': contentknn_itemBased_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_contentknn_itemBased])], ignore_index=True)


In [ ]:
result_df

##### ContentLSTM - SVD Untuned

In [ ]:
# Hybrid Content Based - SVD Untuned
content_svdUntuned_model = HybridAlgorithm([ContentLSTM.fit(trainset), svdUntuned_model], [0.5, 0.5])
content_svdUntuned_rmse, content_svdUntuned_mae, content_svdUntuned_train_time, content_svdUntuned_test_time = train_and_test_model(content_svdUntuned_model, trainset, testset)
result_content_svdUntuned = {
    'Algorithm': 'Hybrid Content Based LSTM - SVD Untuned',
    'RMSE': content_svdUntuned_rmse,
    'MAE': content_svdUntuned_mae,
    'Training Time (s)': content_svdUntuned_train_time,
    'Testing Time (s)': content_svdUntuned_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_content_svdUntuned])], ignore_index=True)


In [ ]:
result_df

##### ContentKNN - SVD Untuned

In [ ]:
# Hybrid Content Based KNN - SVD Untuned
contentknn_svdUntuned_model = HybridAlgorithm([ContentKNN.fit(trainset), svdUntuned_model], [0.5, 0.5])
contentknn_svdUntuned_rmse, contentknn_svdUntuned_mae, contentknn_svdUntuned_train_time, contentknn_svdUntuned_test_time = train_and_test_model(contentknn_svdUntuned_model, trainset, testset)
result_contentknn_svdUntuned = {
    'Algorithm': 'Hybrid Content Based KNN - SVD Untuned',
    'RMSE': contentknn_svdUntuned_rmse,
    'MAE': contentknn_svdUntuned_mae,
    'Training Time (s)': contentknn_svdUntuned_train_time,
    'Testing Time (s)': contentknn_svdUntuned_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_contentknn_svdUntuned])], ignore_index=True)


In [ ]:
result_df

##### ContentLSTM - SVD Tuned

In [ ]:
# Hybrid Content Based LSTM - SVD Tuned
content_svdTuned_model = HybridAlgorithm([ContentLSTM.fit(trainset), svdTuned_model], [0.5, 0.5])
content_svdTuned_rmse, content_svdTuned_mae, content_svdTuned_train_time, content_svdTuned_test_time = train_and_test_model(content_svdTuned_model, trainset, testset)
result_content_svdTuned = {
    'Algorithm': 'Hybrid Content Based LSTM - SVD Tuned',
    'RMSE': content_svdTuned_rmse,
    'MAE': content_svdTuned_mae,
    'Training Time (s)': content_svdTuned_train_time,
    'Testing Time (s)': content_svdTuned_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_content_svdTuned])], ignore_index=True)


In [ ]:
result_df

##### ContentKNN - SVD Tuned

In [ ]:
# Hybrid Content Based KNN - SVD Tuned
contentknn_svdTuned_model = HybridAlgorithm([ContentKNN.fit(trainset), svdTuned_model], [0.5, 0.5])
contentknn_svdTuned_rmse, contentknn_svdTuned_mae, contentknn_svdTuned_train_time, contentknn_svdTuned_test_time = train_and_test_model(contentknn_svdTuned_model, trainset, testset)
result_contentknn_svdTuned = {
    'Algorithm': 'Hybrid Content Based KNN - SVD Tuned',
    'RMSE': contentknn_svdTuned_rmse,
    'MAE': contentknn_svdTuned_mae,
    'Training Time (s)': contentknn_svdTuned_train_time,
    'Testing Time (s)': contentknn_svdTuned_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_contentknn_svdTuned])], ignore_index=True)


In [ ]:
result_df

##### ContentLSTM - Neural

In [ ]:
# Hybrid Content Based LSTM - Neural
content_neural_model = HybridAlgorithm([ContentLSTM.fit(trainset), ncf_model], [0.5, 0.5])
content_neural_rmse, content_neural_mae, content_neural_train_time, content_neural_test_time = train_and_test_model(content_neural_model, trainset, testset)
result_content_neural = {
    'Algorithm': 'Hybrid Content Based LSTM - Neural',
    'RMSE': content_neural_rmse,
    'MAE': content_neural_mae,
    'Training Time (s)': content_neural_train_time,
    'Testing Time (s)': content_neural_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_content_neural])], ignore_index=True)


In [ ]:
result_df

##### ContentKNN - Neural

In [ ]:
# Hybrid Content Based KNN - Neural
contentknn_neural_model = HybridAlgorithm([ContentKNN.fit(trainset), ncf_model], [0.5, 0.5])
contentknn_neural_rmse, contentknn_neural_mae, contentknn_neural_train_time, contentknn_neural_test_time = train_and_test_model(contentknn_neural_model, trainset, testset)
result_contentknn_neural = {
    'Algorithm': 'Hybrid Content Based KNN - Neural',
    'RMSE': contentknn_neural_rmse,
    'MAE': contentknn_neural_mae,
    'Training Time (s)': contentknn_neural_train_time,
    'Testing Time (s)': contentknn_neural_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_contentknn_neural])], ignore_index=True)


In [ ]:
result_df

##### SVD Untuned - SVD Tuned

In [ ]:
# Hybrid SVD Untuned - SVD Tuned
svdUntuned_svdTuned_model = HybridAlgorithm([svdUntuned_model, svdTuned_model], [0.5, 0.5])
svdUntuned_svdTuned_rmse, svdUntuned_svdTuned_mae, svdUntuned_svdTuned_train_time, svdUntuned_svdTuned_test_time = train_and_test_model(svdUntuned_svdTuned_model, trainset, testset)
result_svdUntuned_svdTuned = {
    'Algorithm': 'Hybrid SVD Untuned - SVD Tuned',
    'RMSE': svdUntuned_svdTuned_rmse,
    'MAE': svdUntuned_svdTuned_mae,
    'Training Time (s)': svdUntuned_svdTuned_train_time,
    'Testing Time (s)': svdUntuned_svdTuned_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_svdUntuned_svdTuned])], ignore_index=True)


In [ ]:
result_df

##### SVD Tuned - User Based

In [ ]:
# Hybrid SVD Tuned - User Based
svdTuned_userBased_model = HybridAlgorithm([svdTuned_model, UserKNNmeans.fit(trainset)], [0.5, 0.5])
svdTuned_userBased_rmse, svdTuned_userBased_mae, svdTuned_userBased_train_time, svdTuned_userBased_test_time = train_and_test_model(svdTuned_userBased_model, trainset, testset)
result_svdTuned_userBased = {
    'Algorithm': 'Hybrid SVD Tuned - User Based',
    'RMSE': svdTuned_userBased_rmse,
    'MAE': svdTuned_userBased_mae,
    'Training Time (s)': svdTuned_userBased_train_time,
    'Testing Time (s)': svdTuned_userBased_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_svdTuned_userBased])], ignore_index=True)


In [ ]:
result_df

##### SVD Tuned - Item Based

In [ ]:
# Hybrid SVD Tuned - Item Based
svdTuned_itemBased_model = HybridAlgorithm([svdTuned_model, ItemKNNmeans.fit(trainset)], [0.5, 0.5])
svdTuned_itemBased_rmse, svdTuned_itemBased_mae, svdTuned_itemBased_train_time, svdTuned_itemBased_test_time = train_and_test_model(svdTuned_itemBased_model, trainset, testset)
result_svdTuned_itemBased = {
    'Algorithm': 'Hybrid SVD Tuned - Item Based',
    'RMSE': svdTuned_itemBased_rmse,
    'MAE': svdTuned_itemBased_mae,
    'Training Time (s)': svdTuned_itemBased_train_time,
    'Testing Time (s)': svdTuned_itemBased_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_svdTuned_itemBased])], ignore_index=True)


In [ ]:
result_df

##### SVD Tuned - Neural

In [ ]:
# Hybrid SVD Tuned - Neural
svdTuned_neural_model = HybridAlgorithm([svdTuned_model, ncf_model], [0.5, 0.5])
svdTuned_neural_rmse, svdTuned_neural_mae, svdTuned_neural_train_time, svdTuned_neural_test_time = train_and_test_model(svdTuned_neural_model, trainset, testset)
result_svdTuned_neural = {
    'Algorithm': 'Hybrid SVD Tuned - Neural',
    'RMSE': svdTuned_neural_rmse,
    'MAE': svdTuned_neural_mae,
    'Training Time (s)': svdTuned_neural_train_time,
    'Testing Time (s)': svdTuned_neural_test_time
}
result_df = pd.concat([result_df, pd.DataFrame([result_svdTuned_neural])], ignore_index=True)


In [ ]:
result_df

In [ ]:
# result_df.to_excel('result2.xlSsx', index=False)